In [1]:
import re
from collections import deque

In [2]:
# global variables
obstacles = []
nets = {}
used_paths = set() 
ROWS = COLS = VIA_COST = NON_PREF_COST = 0

<h2> Parsing </h2>

In [4]:
def parse_input_file(file_path):
    
    with open(file_path) as f:
        global ROWS, COLS, VIA_COST, NON_PREF_COST
        values = f.readline().strip()
        ROWS, COLS, VIA_COST, NON_PREF_COST = map(int, values.split(','))

        for line in f:
            line = line.strip()
            if line.startswith("OBS"):
                obstacles.append(tuple(map(int, re.findall(r"\d+", line))))
            elif line.startswith("net"):
                net_name, coords = line.split(' ', 1)
                nets[net_name] = [tuple(map(int, re.findall(r"\d+", coord))) for coord in coords.split(') (')]

In [5]:
def print_input_file_info():
    print(f"Rows: {ROWS}\nCols: {COLS}\nVia Cost: {VIA_COST}\nNon-preferred Direction Cost: {NON_PREF_COST}\n")
    
    print("Obstacles:")
    for obs in obstacles:
        print(f"{obs}")
    
    print("\nNets:")
    for net, coords in nets.items():
        print(f"{net}: {coords}")

In [6]:
parse_input_file('input.txt')
print_input_file_info()

Rows: 5
Cols: 5
Via Cost: 5
Non-preferred Direction Cost: 2

Obstacles:

Nets:
net1: [(0, 1, 0), (0, 2, 1)]
net2: [(0, 1, 4), (0, 2, 3)]
net3: [(0, 4, 0), (0, 0, 4)]


<h2> Routing </h2>

In [8]:
def floodfill(start, end, visited_global):
    queue = deque([(start[1], start[2], None)])  # (x, y, prev_direction)
    
    # Create a local copy of the visited_global for this net's routing
    visited_local = set()

    grid = [[float('inf')] * COLS for _ in range(ROWS)]  
    grid[start[1]][start[2]] = 0

    parent = {}  # (x, y) -> (prev_x, prev_y)

    DIRECTIONS = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    while queue:
        x, y, prev_direction = queue.popleft()

        # Skip if the cell is already visited (global visited or local visited)
        if (x, y) in visited_local or (x, y) in visited_global or (x, y) in obstacles:
            continue
        visited_local.add((x, y))  # Mark as visited locally

        if (x, y) == (end[1], end[2]):
            path = [(end[1], end[2])]
            while path[-1] in parent:
                path.append(parent[path[-1]])
            path.reverse()

            # Add the path to visited_global
            for (px, py) in path:
                visited_global.add((px, py))  # Mark cells that were part of the path

            return grid[end[1]][end[2]], path  # cost, path

        for dx, dy in DIRECTIONS:
            nx, ny = x + dx, y + dy

            if 0 <= nx < ROWS and 0 <= ny < COLS:
                if (nx, ny) in visited_local or (nx, ny) in visited_global or (nx, ny) in obstacles:
                    continue

                if prev_direction is not None:
                    if (dx == 0 and prev_direction == 'H') or (dy == 0 and prev_direction == 'V'):
                        new_cost = grid[x][y] + VIA_COST + 1
                    else:
                        new_cost = grid[x][y] + 1
                else:
                    new_cost = grid[x][y] + 1

                if new_cost < grid[nx][ny]:
                    grid[nx][ny] = new_cost
                    parent[(nx, ny)] = (x, y)
                    queue.append((nx, ny, 'H' if dy == 0 else 'V'))

    return -1, []  # No path found


In [9]:
def route_nets():
    routed_nets = {}
    visited_global = set()  # Keep track of cells that are part of any path

    for net_name, coords in nets.items():
        start, end = coords
        print(f"Routing {net_name} from {start} to {end}")
        
        cost, path = floodfill(start, end, visited_global)
        
        if cost != -1:
            print(f"{net_name} cost: {cost}")
            routed_nets[net_name] = path
        else:
            print(f"No route found for {net_name}.")
        print()

    write_output(routed_nets)  # Write the result to the output file


In [10]:
def write_output(routed_nets):
    with open("output.txt", "w") as f:
        for net_name, path in routed_nets.items():
            line = f"{net_name} " + " ".join(f"({x}, {y})" for (x, y) in path)
            f.write(line + "\n")

In [11]:
route_nets()

Routing net1 from (0, 1, 0) to (0, 2, 1)
net1 cost: 7

Routing net2 from (0, 1, 4) to (0, 2, 3)
net2 cost: 7

Routing net3 from (0, 4, 0) to (0, 0, 4)
net3 cost: 18

